This example requires you to get an OPENAI_API_KEY. 


In [1]:
# Jupyter-friendly version of ZRetriever usage

import nest_asyncio
import asyncio
from zmongo_retriever.zmongo_toolbag.zmongo import ZMongo
from zmongo_retriever.zmongo_toolbag.zretriever import ZRetriever

# Allow nested event loops for Jupyter compatibility
nest_asyncio.apply()

# Define the async logic directly
async def run_zretriever_demo():
    repo = ZMongo()
    retriever = ZRetriever(repository=repo, max_tokens_per_set=4096, chunk_size=512)

    # Replace with real collection name and document IDs
    collection_name = 'documents'
    document_ids = ['67e5ba645f74ae46ad39929d', '67ef0bd71a349c7c108331a6']

    documents = await retriever.invoke(collection=collection_name, object_ids=document_ids, page_content_key='text')

    for idx, doc_set in enumerate(documents):
        print(f"\nDocument Set {idx + 1}:")
        for doc in doc_set:
            print(f"Metadata: {doc.metadata}")
            print(f"Content: {doc.page_content[:200]}...\n")

# Run the async function in Jupyter
await run_zretriever_demo()


DEBUG:pymongo.topology:{"topologyId": {"$oid": "67efe198ac294b6cae3a87cf"}, "message": "Starting topology monitoring"}
DEBUG:pymongo.topology:{"topologyId": {"$oid": "67efe198ac294b6cae3a87cf"}, "previousDescription": "<TopologyDescription id: 67efe198ac294b6cae3a87cf, topology_type: Unknown, servers: []>", "newDescription": "<TopologyDescription id: 67efe198ac294b6cae3a87cf, topology_type: Unknown, servers: [<ServerDescription ('127.0.0.1', 27017) server_type: Unknown, rtt: None>]>", "message": "Topology description changed"}
DEBUG:pymongo.topology:{"topologyId": {"$oid": "67efe198ac294b6cae3a87cf"}, "serverHost": "127.0.0.1", "serverPort": 27017, "message": "Starting server monitoring"}
/media/overlordx/DATA/PycharmProjects/zvirtualtour/zmongo_retriever/zmongo_toolbag/zretriever.py:32: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package


Document Set 1:
Metadata: {'source': 'mongodb', 'database_name': 'unknown', 'collection_name': 'unknown', 'document_id': "{'$oid': '67e5ba645f74ae46ad39929d'}"}
Content: 6 555
4444
333
22
1
0000000000
1
22
333
4444
asdf
GREG LANG AND ELLA L. LANG 
 VS. 
 MARGARET M. BEAULAC f/k/a 
 MARGARET M. ESPAILLAT 
 CASE NO.: 2022-CA-006190-O 
 TABLE OF CONTENTS ...

Metadata: {'source': 'mongodb', 'database_name': 'unknown', 'collection_name': 'unknown', 'document_id': "{'$oid': '67e5ba645f74ae46ad39929d'}"}
Content: Plaintiff’s Motion for Summary Judgement 
 I. INTRODUCTION AND UNDISPUTED FACTS 
 Johnson v. Davis  , 480 So. 2d 625, 629 (Fla. 1985)  5 
 Winfield Investments, LLC v. Pascal-Gaston Investments, LL...

Metadata: {'source': 'mongodb', 'database_name': 'unknown', 'collection_name': 'unknown', 'document_id': "{'$oid': '67e5ba645f74ae46ad39929d'}"}
Content: Matsushita Electric Industrial Co. v. Zenith Radio Corp., 475 U.S. 574 (1986)  10 
 Riti Fin., LLC v. Patel, 386 So.3d 1058, 1060 

Set your variables

Use ZMongoRetriever to split the text from the page_content_field into LangChain Documents

If the output from above == [] then you have a problem with your variables.

Use LlamaCPP to summarize the chunked output.

In [4]:
# Jupyter cell: Combine ZRetriever with LlamaModel to query MongoDB and generate LLM completions

import os
import nest_asyncio
import asyncio
from dotenv import load_dotenv
from zmongo_retriever.zmongo_toolbag.zmongo import ZMongo
from zmongo_retriever.zmongo_toolbag.zretriever import ZRetriever
from llama_cpp import Llama

# Load environment variables and allow nested asyncio
load_dotenv()
nest_asyncio.apply()

class LlamaModel:
    def __init__(self):
        self.model_path = os.getenv(
            "GGUF_MODEL_PATH",

        )
        self.n_ctx = int(os.getenv("N_CTX", 512))
        self.n_batch = int(os.getenv("N_BATCH", 126))
        self.llm = None
        self.load_model()

    def load_model(self):
        print("Loading model...")
        self.llm = Llama(
            model_path=self.model_path,
            n_ctx=self.n_ctx,
            n_batch=self.n_batch
        )
        print("Model loaded successfully.")

    def generate_prompt_from_template(self, user_input: str) -> str:
        return (
            "<|im_start|>system\n"
            "You are a helpful chatbot.<|im_end|>\n"
            "<|im_start|>user\n"
            f"{user_input}<|im_end|>"
        )

    def generate_text(self, prompt: str, max_tokens: int = 256, temperature: float = 0.1, top_p: float = 0.5, echo: bool = False, stop: list = None) -> str:
        stop = stop or ["#"]
        output = self.llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            echo=echo,
            stop=stop,
        )
        return output["choices"][0]["text"].strip()

# Main async function to retrieve and summarize using Llama
async def run_llama_zretriever_pipeline():
    repo = ZMongo()
    retriever = ZRetriever(repository=repo, max_tokens_per_set=4096, chunk_size=512)
    collection_name = 'documents'
    document_ids = ['67e5ba645f74ae46ad39929d', '67ef0bd71a349c7c108331a6']

    documents = await retriever.invoke(collection=collection_name, object_ids=document_ids, page_content_key='text')

    llama_model = LlamaModel()

    for idx, doc_set in enumerate(documents):
        combined_text = "\n\n".join(doc.page_content for doc in doc_set)
        print(f"\nDocument Set {idx + 1} (retrieved {len(doc_set)} chunks):")
        prompt = llama_model.generate_prompt_from_template(
            f"Summarize the following legal content for a law student:\n{combined_text[:1000]}"
        )
        summary = llama_model.generate_text(prompt, max_tokens=512)
        print("\nSummary:")
        print(summary)

# Run in notebook
await run_llama_zretriever_pipeline()


DEBUG:pymongo.topology:{"topologyId": {"$oid": "67efe408ac294b6cae3a87d2"}, "message": "Starting topology monitoring"}
DEBUG:pymongo.topology:{"topologyId": {"$oid": "67efe408ac294b6cae3a87d2"}, "previousDescription": "<TopologyDescription id: 67efe408ac294b6cae3a87d2, topology_type: Unknown, servers: []>", "newDescription": "<TopologyDescription id: 67efe408ac294b6cae3a87d2, topology_type: Unknown, servers: [<ServerDescription ('127.0.0.1', 27017) server_type: Unknown, rtt: None>]>", "message": "Topology description changed"}
DEBUG:pymongo.topology:{"topologyId": {"$oid": "67efe408ac294b6cae3a87d2"}, "serverHost": "127.0.0.1", "serverPort": 27017, "message": "Starting server monitoring"}
DEBUG:pymongo.connection:{"clientId": {"$oid": "67efe408ac294b6cae3a87d2"}, "message": "Connection pool created", "serverHost": "127.0.0.1", "serverPort": 27017, "maxPoolSize": 200}
DEBUG:pymongo.serverSelection:{"message": "Server selection started", "selector": "Primary()", "operation": "find", "top

Loading model...


llm_load_tensors:        CPU buffer size =  3917.87 MiB
..................................................................................................
llama_new_context_with_model: n_ctx      = 4096
llama_new_context_with_model: n_batch    = 4096
llama_new_context_with_model: n_ubatch   = 512
llama_new_context_with_model: flash_attn = 0
llama_new_context_with_model: freq_base  = 10000.0
llama_new_context_with_model: freq_scale = 1
llama_kv_cache_init:        CPU KV buffer size =   512.00 MiB
llama_new_context_with_model: KV self size  =  512.00 MiB, K (f16):  256.00 MiB, V (f16):  256.00 MiB
llama_new_context_with_model:        CPU  output buffer size =     0.12 MiB
llama_new_context_with_model:        CPU compute buffer size =   296.01 MiB
llama_new_context_with_model: graph nodes  = 1030
llama_new_context_with_model: graph splits = 1
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | 

Model loaded successfully.

Document Set 1 (retrieved 28 chunks):



llama_print_timings:        load time =   28255.90 ms
llama_print_timings:      sample time =       0.03 ms /     1 runs   (    0.03 ms per token, 30303.03 tokens per second)
llama_print_timings: prompt eval time =   28255.45 ms /   543 tokens (   52.04 ms per token,    19.22 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   28256.05 ms /   544 tokens
Llama.generate: 46 prefix-match hit, remaining 349 prompt tokens to eval


...


Summary:
eated debris. The roof is in need of repair and replacement. 
 2) The exterior walls of the house are in need of repair and repainting. The paint is peeling 
 and flaking in several locations. 
 3) The driveway is cracked and in need of repair. 
 4) The fence is in need of repair and replacement. 
 5) The front door is in need of repair and replacement. 
 6) The back door is in need of repair and replacement. 
 7) The windows are in need of repair and replacement. 
 8) The siding is in need of repair and replacement. 
 9) The chimney is in need of repair and replacement. 
 10) The deck is in need of repair and replacement. 
 11) The pool fence is in need of repair and replacement. 
 12) The pool cover is in need of repair and replacement. 
 13) The pool heater is in need of repair and replacement. 
 14) The pool pump is in need of repair and replacement. 
 15) The pool filter is in need of repair and replacement. 
 16) The pool light is in need of repair and replacement. 
 17


llama_print_timings:        load time =   28255.90 ms
llama_print_timings:      sample time =       0.03 ms /     1 runs   (    0.03 ms per token, 33333.33 tokens per second)
llama_print_timings: prompt eval time =   20659.71 ms /   381 tokens (   54.22 ms per token,    18.44 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   20659.66 ms /   382 tokens
Llama.generate: 46 prefix-match hit, remaining 422 prompt tokens to eval



Summary:


Document Set 15 (retrieved 25 chunks):



llama_print_timings:        load time =   28255.90 ms
llama_print_timings:      sample time =       0.03 ms /     1 runs   (    0.03 ms per token, 35714.29 tokens per second)
llama_print_timings: prompt eval time =   21231.49 ms /   422 tokens (   50.31 ms per token,    19.88 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   21231.85 ms /   423 tokens



Summary:



In [5]:
# Jupyter cell: Combine ZRetriever with OpenAIModel to query MongoDB and generate completions

import os
import nest_asyncio
import asyncio
from bson import ObjectId
from dotenv import load_dotenv
from zmongo_retriever.zmongo_toolbag.zmongo import ZMongo
from zmongo_retriever.zmongo_toolbag.zretriever import ZRetriever
import openai

# Load environment variables and allow nested asyncio
load_dotenv()
nest_asyncio.apply()
openai.api_key = os.getenv("OPENAI_API_KEY_APP")

class OpenAIModel:
    def __init__(self):
        self.model = os.getenv("OPENAI_TEXT_MODEL", "gpt-3.5-turbo-instruct")
        self.max_tokens = int(os.getenv("DEFAULT_MAX_TOKENS", 256))
        self.temperature = float(os.getenv("DEFAULT_TEMPERATURE", 0.7))
        self.top_p = float(os.getenv("DEFAULT_TOP_P", 0.95))

    async def _call_openai(self, prompt: str, max_tokens=None, temperature=None, top_p=None, stop=None, echo=False) -> str:
        try:
            response = await asyncio.to_thread(openai.completions.create,
                model=self.model,
                prompt=prompt,
                max_tokens=max_tokens or self.max_tokens,
                temperature=temperature or self.temperature,
                top_p=top_p or self.top_p,
                stop=stop,
                echo=echo,
            )
            return response.choices[0].text.strip()
        except Exception as e:
            return f"[OpenAI Error] {e}"

    async def summarize_text(self, text: str) -> str:
        prompt = f"Summarize the following for a legal researcher:\n\n{text}\n\nSummary:"
        return await self._call_openai(prompt, max_tokens=200)

# Main async function to retrieve and summarize using OpenAI
async def run_openai_zretriever_pipeline():
    repo = ZMongo()
    retriever = ZRetriever(repository=repo, max_tokens_per_set=4096, chunk_size=512)
    collection_name = 'documents'
    document_ids = ['67e5ba645f74ae46ad39929d', '67ef0bd71a349c7c108331a6']

    documents = await retriever.invoke(collection=collection_name, object_ids=document_ids, page_content_key='text')
    openai_model = OpenAIModel()

    for idx, doc_set in enumerate(documents):
        combined_text = "\n\n".join(doc.page_content for doc in doc_set)
        print(f"\nDocument Set {idx + 1} (retrieved {len(doc_set)} chunks):")
        summary = await openai_model.summarize_text(combined_text[:1000])
        print("\nSummary:")
        print(summary)

# Run in notebook
await run_openai_zretriever_pipeline()


DEBUG:pymongo.topology:{"topologyId": {"$oid": "67efe7b4ac294b6cae3a87d3"}, "message": "Starting topology monitoring"}
DEBUG:pymongo.topology:{"topologyId": {"$oid": "67efe7b4ac294b6cae3a87d3"}, "previousDescription": "<TopologyDescription id: 67efe7b4ac294b6cae3a87d3, topology_type: Unknown, servers: []>", "newDescription": "<TopologyDescription id: 67efe7b4ac294b6cae3a87d3, topology_type: Unknown, servers: [<ServerDescription ('127.0.0.1', 27017) server_type: Unknown, rtt: None>]>", "message": "Topology description changed"}
DEBUG:pymongo.topology:{"topologyId": {"$oid": "67efe7b4ac294b6cae3a87d3"}, "serverHost": "127.0.0.1", "serverPort": 27017, "message": "Starting server monitoring"}
DEBUG:pymongo.connection:{"clientId": {"$oid": "67efe7b4ac294b6cae3a87d3"}, "message": "Connection pool created", "serverHost": "127.0.0.1", "serverPort": 27017, "maxPoolSize": 200}
DEBUG:pymongo.serverSelection:{"message": "Server selection started", "selector": "Primary()", "operation": "find", "top


Document Set 1 (retrieved 28 chunks):


DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x750911593a90>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 04 Apr 2025 14:07:51 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'access-control-expose-headers', b'X-Request-ID'), (b'Cache-Control', b'no-cache, must-revalidate'), (b'openai-model', b'gpt-3.5-turbo-instruct:20230824-v2'), (b'openai-organization', b'user-2wnkfo6olprbzoagrydr5rhm'), (b'openai-processing-ms', b'170


Summary:
This document is a case between Greg Lang and Ella L. Lang against Margaret M. Beaulac, formerly known as Margaret M. Espailiat. The case number is 2022-CA-006190-O and includes a table of contents, defendant's amended verified motion for summary judgment, plaintiffs' response in opposition to defendant's motion for summary judgment, plaintiffs' motion for summary judgment, defendant's response in opposition to plaintiff's motion for summary judgment, and relevant case law. The case involves a motion for summary judgment, and relevant case law includes Johnson v. Davis, Winfield Investments LLC v. Pascal-Gaston Investments LLC, Butler v. Yusem, Celotex Corp. v. Catrett, Anderson v. Liberty Lobby Inc., and Matsushita Electric Industrial Co. v. Z.

Document Set 2 (retrieved 31 chunks):


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 04 Apr 2025 14:07:53 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'access-control-expose-headers', b'X-Request-ID'), (b'Cache-Control', b'no-cache, must-revalidate'), (b'openai-model', b'gpt-3.5-turbo-instruct:20230824-v2'), (b'openai-organization', b'user-2wnkfo6olprbzoagrydr5rhm'), (b'openai-processing-ms', b'1406'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'via', b'envoy-router-868dc8d978-tnzgz'), (b'x-envoy-upstream-service-time', b'1344'), (b'x-ratelimit-limit-requests', b'3500'), (b'x-ratelimit-limit-tokens', b'90000'), (b'x-ratelimit-remaining-requests', b'3499'), (b'x-ratelimit-remaining-tokens', b'89731'), (b'x-ratelimit-reset-requests', b'17ms'), (b'x-ratelimit-reset-tokens', b'

...


Summary:
This report provides a summary of a property inspection for 4901 Sudbury Court in Orlando, Florida. The report is organized into different categories, including general, exterior, roofing, attic space, electrical, plumbing systems and water heaters, heating and air conditioning, bathrooms, kitchen and breakfast nook, interior, and pool/spa. The report recommends reading the complete report for additional information and suggests seeking advice from a real estate agent or attorney for any items that may be negotiable under the real estate purchase contract. The report notes several exterior defects, including roof eave rebuilding and gutter issues.

Document Set 14 (retrieved 29 chunks):


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 04 Apr 2025 14:08:14 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'access-control-expose-headers', b'X-Request-ID'), (b'Cache-Control', b'no-cache, must-revalidate'), (b'openai-model', b'gpt-3.5-turbo-instruct:20230824-v2'), (b'openai-organization', b'user-2wnkfo6olprbzoagrydr5rhm'), (b'openai-processing-ms', b'671'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'via', b'envoy-router-canary-54f4d9c485-lz9q7'), (b'x-envoy-upstream-service-time', b'655'), (b'x-ratelimit-limit-requests', b'3500'), (b'x-ratelimit-limit-tokens', b'90000'), (b'x-ratelimit-remaining-requests', b'3499'), (b'x-ratelimit-remaining-tokens', b'89734'), (b'x-ratelimit-reset-requests', b'17ms'), (b'x-ratelimit-reset-tokens


Summary:
The question is whether there is evidence of an original note in a legal case. The only evidence found is a notice of filing by a previous lawyer, and the word of that lawyer that the document is original. The researcher is questioning whether anyone has actually retrieved the document from the file.

Document Set 15 (retrieved 25 chunks):


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 04 Apr 2025 14:08:15 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'access-control-expose-headers', b'X-Request-ID'), (b'Cache-Control', b'no-cache, must-revalidate'), (b'openai-model', b'gpt-3.5-turbo-instruct:20230824-v2'), (b'openai-organization', b'user-2wnkfo6olprbzoagrydr5rhm'), (b'openai-processing-ms', b'760'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'via', b'envoy-router-c5c594c44-tv784'), (b'x-envoy-upstream-service-time', b'745'), (b'x-ratelimit-limit-requests', b'3500'), (b'x-ratelimit-limit-tokens', b'90000'), (b'x-ratelimit-remaining-requests', b'3499'), (b'x-ratelimit-remaining-tokens', b'89734'), (b'x-ratelimit-reset-requests', b'17ms'), (b'x-ratelimit-reset-tokens', b'176


Summary:
This document is a transcript of a legal proceeding involving a court reporting company and an attorney. The attorney, Mr. Iriye, is asking a question to the court reporter and there is a brief pause during which the proceedings go off the record. The document being discussed is a Limited Power of Attorney.
